In [2]:
import json
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import pandas as pd
with open('train.json') as json_data:
    d = json.load(json_data)

In [3]:

ingredients = []
cuisines = []

In [4]:
for i in range(0,len(d)):
    cuisines.append(d[i]["cuisine"].lower())
    for ingredient in d[i]["ingredients"]:
        ingredients.append(ingredient.lower())

In [5]:
cuisines = list(set(cuisines))
ingredients = list(set(ingredients))
print(len(d))
print(len(ingredients))
print(len(cuisines))
print(cuisines)


39774
6703
20
['thai', 'russian', 'irish', 'southern_us', 'french', 'cajun_creole', 'filipino', 'jamaican', 'italian', 'british', 'brazilian', 'korean', 'chinese', 'greek', 'mexican', 'japanese', 'vietnamese', 'moroccan', 'spanish', 'indian']


In [6]:
ingredient_dict = {ingredients[i] : i for i in range(0,len(ingredients))}
cuisine_dict = {cuisines[i] : i for i in range(0,len(cuisines))}
print(ingredient_dict)
print(cuisine_dict)

{'fat-free parmesan cheese': 0, 'swerve sweetener': 3293, 'hero rolls': 3294, 'ox tongue': 3295, 'cannellini beans': 3297, 'pecan pie': 1, 'chocolate candy': 3298, 'bbq seasoning': 2, 'frozen onion rings': 6453, 'gingersnap': 3, 'kabocha squash': 4, 'steamed white rice': 3301, 'gray salt': 7, 'smoked mozzarella': 6, 'camembert': 3292, 'baby goat': 3304, 'progresso black beans': 8, 'shoepeg corn': 3893, 'shiso': 9, 'wheat crackers': 10, 'buffalo': 3306, 'mo hanh': 3307, 'bean dip': 3308, 'fish balls': 11, 'mccormick parsley flakes': 5547, 'french lentils': 3309, 'knorr tomato bouillon with chicken flavor cube': 3296, 'oscar mayer cotto salami': 3310, 'ganache': 12, 'buttercup squash': 13, 'truvía® baking blend': 14, 'cashew nuts': 15, 'low salt chicken broth': 3312, 'single crust pie': 16, 'refrigerated fettuccine': 3313, 'oil': 17, 'brown mustard': 3314, 'white corn syrup': 5549, 'reduced fat mozzarella': 19, 'gluten-free pie crust': 3315, 'coconut oil': 20, 'fruit cocktail': 3316, 'pa

In [7]:
train_data = np.array(np.zeros([len(d), (2 + len(ingredients))]))
print(train_data.shape)
train_labels = pd.DataFrame(np.array(np.zeros([len(d),1])))

(39774, 6705)


In [8]:
for i in range(0,len(d)):
    train_data[i][0] = d[i]["id"]
    train_labels.iloc[i] = d[i]["cuisine"].lower()
    for ingredient in d[i]["ingredients"]:
        train_data[i][2 + ingredient_dict[ingredient.lower()]] = 1

In [9]:
with open('test.json') as json_data:
    t = json.load(json_data)


In [10]:
test_data = np.array(np.zeros([len(t), (1 + len(ingredients))]))
for i in range(0,len(t)):
    test_data[i][0] = t[i]["id"]
    for ingredient in d[i]["ingredients"]:
        test_data[i][1 + ingredient_dict[ingredient.lower()]] = 1


In [11]:
lgr = LogisticRegression()
lgr_scores = cross_val_score(lgr, train_data[:,2:], train_labels[:].as_matrix().reshape(len(train_data)), cv=3)
print(lgr_scores)
lgr.fit(train_data[:,2:], train_labels[:].as_matrix().reshape(len(train_data)))
pred_labels = pd.DataFrame(lgr.predict(test_data[:,1:]))
print(pred_labels.shape)
print(pred_labels)
ids = pd.DataFrame(test_data[:,0], dtype=int)
print(ids.shape)
results = pd.concat([ids,pred_labels], axis=1)
print(results.shape)
print(results)
#results.to_csv("results.csv", index=False, header=("id", "cuisine"))

[ 0.7749378   0.77373859  0.77897676]
(9944, 1)
                 0
0            greek
1      southern_us
2         filipino
3           indian
4           indian
5         jamaican
6          spanish
7          italian
8          mexican
9          italian
10         italian
11         chinese
12         italian
13         mexican
14         italian
15          indian
16           greek
17         italian
18      vietnamese
19         chinese
20            thai
21         mexican
22     southern_us
23            thai
24         italian
25         chinese
26         mexican
27         italian
28         chinese
29         mexican
...            ...
9914        indian
9915       chinese
9916       italian
9917        indian
9918     brazilian
9919       mexican
9920         greek
9921       italian
9922   southern_us
9923        indian
9924       british
9925       mexican
9926      filipino
9927       mexican
9928         irish
9929   southern_us
9930       chinese
9931       italian
99

In [16]:
print (pred_labels)

                 0
0            greek
1      southern_us
2         filipino
3           indian
4           indian
5         jamaican
6          spanish
7          italian
8          mexican
9          italian
10         italian
11         chinese
12         italian
13         mexican
14         italian
15          indian
16           greek
17         italian
18      vietnamese
19         chinese
20            thai
21         mexican
22     southern_us
23            thai
24         italian
25         chinese
26         mexican
27         italian
28         chinese
29         mexican
...            ...
9914        indian
9915       chinese
9916       italian
9917        indian
9918     brazilian
9919       mexican
9920         greek
9921       italian
9922   southern_us
9923        indian
9924       british
9925       mexican
9926      filipino
9927       mexican
9928         irish
9929   southern_us
9930       chinese
9931       italian
9932  cajun_creole
9933       italian
9934      ja

In [19]:
lgr.score(test_data[:,2:], pred_labels)

ValueError: X has 6702 features per sample; expecting 6703